In [1]:
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
import scipy.ndimage
from scipy import fftpack

In [18]:
def SNR(powerspec):
    covar = np.mat(np.cov(powerspec))
    return np.sqrt(np.sum(powerspec @ covar.I @ powerspec.T))

In [6]:
def power1D(image):
    #help from Jessica Lu's radialProfile module (http://www.astrobetter.com/wiki/tiki-index.php?page=python_radial_profiles)
    y, x = np.indices(image.shape)
    center = np.array([(x.max() - x.min()) / 2., (y.max() - y.min()) / 2.])
    
    if image.shape[0]%2 == 0:
        center+=0.5
    
    radii = np.hypot(x - center[0], y - center[1])
    
    sorted_radii_indices = np.argsort(radii.flat)
    sorted_radii = radii.flat[sorted_radii_indices]
    sorted_pixels = image.flat[sorted_radii_indices]

    
    num_bins = 100
    bins = np.linspace(1.0, image.shape[0] / 2., num_bins + 1)
    bins = np.append([0], bins)
    
    bin_weights = np.histogram(sorted_radii, bins)[0]
    bin_edges = np.cumsum(bin_weights)
    pixel_sums = np.cumsum(sorted_pixels, dtype=float)
    bin_totals = pixel_sums[bin_edges[1:] - 1] - pixel_sums[bin_edges[:-1] - 1]
    
    radial_prof = bin_totals/bin_weights[1:]
    
    return bins[1:-1], radial_prof

In [7]:
image_file = fits.open('file:///C:/cygwin64/home/rehg98/WLconv_z1100.00_0580r.fits')
image_data = image_file[0].data
image_data = image_data.astype(float)
smooth_data = scipy.ndimage.filters.gaussian_filter(image_data, 9.75)
F_smooth = fftpack.fftshift(fftpack.fft2(smooth_data))
psd2D_smooth = np.abs(F_smooth)**2

In [8]:
ell_smooth, psd1D_smooth = power1D(psd2D_smooth)

side_map_deg = 3.5
side_map_pix = 2048

ell_smooth = ell_smooth * 360./np.sqrt(side_map_deg)
norm = ((2*np.pi*np.sqrt(side_map_deg)/360.0)**2)/(side_map_pix**2)**2
powspec_smooth = ell_smooth * (ell_smooth + 1)/(2*np.pi) * norm * psd1D_smooth

In [19]:
print(SNR(powspec_smooth))

LinAlgError: singular matrix